# 0.0 Importações

In [24]:
# Bibliotecas

from bs4 import BeautifulSoup
from selenium import webdriver
from functools import reduce
from time import sleep
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 150)
pd.set_option('display.max_rows', 500)

## 0.1 Carregando base de dados

In [46]:
# Carregando base de dados 
base_dados = pd.read_excel(r'C:\Users\Andre\Documents\Via\Base_Dados_Manuais\ParceriasB2B_BaseDados_Automacao.xlsx')

## 0.2 Funções 

In [26]:
# DEFs PARA LIMPEZA DE DADOS NA BASE 

def limpeza_responseTime(base_dyna):
    for n, v in enumerate(base_dyna):
        if 'ms' in str(v):
            base_dyna[n] = float(str(v[:-3]))

        if 'µs' in str(v):
            base_dyna[n] = float('0.' + str(v[:-3]))

        if ' s' in str(v):
            if len(str(v)[:-2]) <= 2:
                base_dyna[n] = float(str(str(v[:-2]) + '000').replace(' ', ''))

            if len(str(v)[:-2]) >= 3:
                base_dyna[n] = float(str(str(v[:-2]) + '0').replace(' ', ''))

    return base_dyna




def limpeza_taxaErros(base_dyna):
    for n, v in enumerate(base_dyna):
        if '%' in str(v):
            try:
                base_dyna[n] = float(str(v[:-2]))
            except:
                pass
    return base_dyna 



def limpeza_CPU(base_dyna):
    for n, v in enumerate(base_dyna):

        if 'µs/req' in str(v):

            base_dyna[n] = float('0.' + str(v[:-6]))

        if 'ms/req' in str(v):

            base_dyna[n] = float(str(v[:-6]))

        if ' s' in str(v):
            try:
                base_dyna[n] = float(str(str(v[:-5]) + '0').replace(' ', ''))
            except:
                pass
            
    return base_dyna



def limpeza_Requests(Processed_requests):
    for i in range(len(Processed_requests)):
        try:

            if 'k /min' in str(Processed_requests.iloc[i]):
                Processed_requests.iloc[i] = str(Processed_requests.iloc[i]).replace('k /min', '')
                if float(Processed_requests.iloc[i][:2]) > 9:
                    Processed_requests.iloc[i] = str(Processed_requests.iloc[i]).replace('.','') + '00'

                if float(Processed_requests.iloc[i][:2]) < 10: 
                    Processed_requests.iloc[i] = str(Processed_requests.iloc[i]).replace('.','') + '0'

            if '/min' in str(Processed_requests.iloc[i]):
                Processed_requests.iloc[i] = str(Processed_requests.iloc[i]).replace('/min', '')

            
            Processed_requests.iloc[i] = str(Processed_requests.iloc[i]).replace(' ', '')
        except:
            pass

    return Processed_requests



#########################################################################################################################
# DEFs PARA SOMA DE TODO O RESULTADO DA PÁGINA  

def soma_valores_responseTime(base_dyna):

    i = 0 
    u = 0
    arrumacao = ''
    num = []

    while True:

        if '-' in str(base_dyna.iloc[i][-3]):

            break
        if int(str(base_dyna.iloc[i][-3][0])) >= 0:

            num.append(str(base_dyna.iloc[i][-3]))
            i+=2


    for n, v in enumerate(num):
        if 'ms' in str(v):
            num[n] = float(str(v[:-3]))

        if 'µs' in str(v):
            num[n] = float('0.' + str(v[:-3]))

        if ' s' in str(v):
            num[n] = float(str(str(v[:-2]) + '0').replace(' ', ''))

    for n, v in enumerate(num):
        u += float(v)

    return u / len(base_dyna['Name'].dropna())


def soma_valores_taxaErros(base_dyna):
    i = 0 
    u = 0
    arrumacao = ''
    num = []

    while True:
        if '-' in str(base_dyna.iloc[i][-3]):
            break
        if int(str(base_dyna.iloc[i][-3][0])) >= 0:
            num.append(str(base_dyna.iloc[i][-3]))
        i+=2


    for n, v in enumerate(num):
        if '%' in str(v):
            num[n] = float(str(v[:-2]))

    for n, v in enumerate(num):
        u += v

    return u / len(base_dyna['Name'].dropna())



def soma_valores_CPU(base_dyna):
    i = 0 
    u = 0
    arrumacao = ''
    num = []

    while True:
        if '-' in str(base_dyna.iloc[i][3]):

            break

        if int(str(base_dyna.iloc[i][3][0])) >= 0:

            num.append(str(base_dyna.iloc[i][3]))
            i+=2


    for n, v in enumerate(num):

        if 'µs/req' in str(v):

            num[n] = float('0.' + str(v[:-6]))

        if 'ms/req' in str(v):

            num[n] = float(str(v[:-6]))

        if ' s' in str(v):

            num[n] = float(str(str(v[:-5]) + '0').replace(' ', ''))


    for n, v in enumerate(num):
        u += v

    return u / len(base_dyna['Name'].dropna())



def soma_valores_Requests(base_dyna):
    Processed_requests = base_dyna['Processed requests'].dropna().copy()
    p = 0

    for i in range(len(Processed_requests)):
        try:

            if 'k /min' in str(Processed_requests.iloc[i]):
                Processed_requests.iloc[i] = str(Processed_requests.iloc[i]).replace('k /min', '')
                if float(Processed_requests.iloc[i][:2]) > 9:
                    Processed_requests.iloc[i] = str(Processed_requests.iloc[i]).replace('.','') + '00'

                if float(Processed_requests.iloc[i][:2]) < 10: 
                    Processed_requests.iloc[i] = str(Processed_requests.iloc[i]).replace('.','') + '0'

            if '/min' in str(Processed_requests.iloc[i]):
                Processed_requests.iloc[i] = str(Processed_requests.iloc[i]).replace('/min', '')

            
            Processed_requests.iloc[i] = str(Processed_requests.iloc[i]).replace(' ', '')
        except:
            pass

        try:
            p += float(Processed_requests.iloc[i])
        except:
            pass    

    return p / len(Processed_requests)

# 1.0 Dados de Acesso

## 1.1 Data e Hora dos Testes

In [47]:
# DATA DO TESTE
data = '16/09/21'

# NÚMERO DE BATERIAS QUE VÃO RODAR NO TESTE
num_bateria = 2

# DATA DO TESTE E HORÁRIO DA PRIMEIRA BATERIA 
data_anterior_1 = 'gtf=2021-07-29T23:00:00-03:00%20to%202021-07-30T06:00:00-03:00'
data_atual_1 = 'gtf=2021-09-17T00:20:00-03:00%20to%202021-09-17T02:30:00-03:00'

# DATA DO TESTE E HORÁRIO DA SEGUNDA BATERIA 
data_anterior_2 = 'gtf=2021-07-29T23:00:00-03:00%20to%202021-07-30T06:00:00-03:00'
data_atual_2 = 'gtf=2021-09-17T03:00:00-03:00%20to%202021-09-17T04:10:00-03:00'

# DATA DO TESTE E HORÁRIO DA TERCEIRA BATERIA 
data_anterior_3 = 'gtf=2021-07-29T23:00:00-03:00%20to%202021-07-30T06:00:00-03:00'
data_atual_3 = 'gtf=2021-09-17T03:50:00-03:00%20to%202021-09-17T04:40:00-03:00'

## 1.2 Acesso ao Dynatrace

In [28]:
# dados de acesso ao Dynatrace
mail_address = 'email@via.com.br'
password = 'senha'
authenticator = 'codigo authenticator VIA'


# acessando Dynatrace
driver = webdriver.Chrome(executable_path = r'C:\Users\Andre\Documents\Via\ChromeDriver\chromedriver.exe')


driver.get('https://sso.dynatrace.com/action/signin#email-verification-page')


sleep(3)
# logando com email
driver.find_element_by_id('email_verify').send_keys(mail_address) 
driver.find_element_by_id('next_button').click()

sleep(2)

# confirmando email
driver.find_element_by_id('i0116').send_keys(mail_address)
driver.find_element_by_id('idSIButton9').click()

sleep(2)

# colocando a senha 
driver.find_element_by_name('passwd').send_keys(password)
driver.find_element_by_id("idSIButton9").click()

sleep(3)

# colocando codigo authenticator
driver.find_element_by_id('idTxtBx_SAOTCC_OTC').send_keys(authenticator)
driver.find_element_by_id('idSubmit_SAOTCC_Continue').click()

sleep(2)

# confirmando acesso
driver.find_element_by_id('idSIButton9').click()

# 2.0 Primeira Bateria do Teste

## 2.3 Imputando Dados de Data e Hora da Primeira Bateria do Teste

In [48]:
# ALTERANDO O LINK COM A DATA E HORA DA PRIMEIRA BATERIA
for i in range(len(base_dados)):
    base_dados['LINK RESPONSE TIME CB'].iloc[i] = str(base_dados['LINK RESPONSE TIME CB'].iloc[i]).replace(data_anterior_1, data_atual_1)
    base_dados['LINK RESPONSE TIME EX'].iloc[i] = str(base_dados['LINK RESPONSE TIME EX'].iloc[i]).replace(data_anterior_1, data_atual_1)
    base_dados['LINK RESPONSE TIME PF'].iloc[i] = str(base_dados['LINK RESPONSE TIME PF'].iloc[i]).replace(data_anterior_1, data_atual_1)


    base_dados['LINK FAILURE RATE CB'].iloc[i] = str(base_dados['LINK FAILURE RATE CB'].iloc[i]).replace(data_anterior_1, data_atual_1)
    base_dados['LINK FAILURE RATE EX'].iloc[i] = str(base_dados['LINK FAILURE RATE EX'].iloc[i]).replace(data_anterior_1, data_atual_1)
    base_dados['LINK FAILURE RATE PF'].iloc[i] = str(base_dados['LINK FAILURE RATE PF'].iloc[i]).replace(data_anterior_1, data_atual_1)

    base_dados['LINK CPU CB'].iloc[i] = str(base_dados['LINK CPU CB'].iloc[i]).replace(data_anterior_1, data_atual_1)
    base_dados['LINK CPU EX'].iloc[i] = str(base_dados['LINK CPU EX'].iloc[i]).replace(data_anterior_1, data_atual_1)
    base_dados['LINK CPU PF'].iloc[i] = str(base_dados['LINK CPU PF'].iloc[i]).replace(data_anterior_1, data_atual_1)

    base_dados['LINK REQUESTS CB'].iloc[i] = str(base_dados['LINK REQUESTS CB'].iloc[i]).replace(data_anterior_1, data_atual_1)
    base_dados['LINK REQUESTS EX'].iloc[i] = str(base_dados['LINK REQUESTS EX'].iloc[i]).replace(data_anterior_1, data_atual_1)
    base_dados['LINK REQUESTS PF'].iloc[i] = str(base_dados['LINK REQUESTS PF'].iloc[i]).replace(data_anterior_1, data_atual_1)


C:\Users\Andre\AppData\Roaming\Python\Python37\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


## 2.4 Rodando Extração de Dados da Primeira Bateria do Teste

In [49]:
# BATERIA 1

# Extraindo dados de RESPONSE TIME, FAILURE RATE, CPU e REQUESTS diretamente do dynatrace 

if 1 <= int(num_bateria): 

    for i in range(len(base_dados)):

        base_dados['BATERIA'][i] = 1
        base_dados['DATA'][i] = data

        # RESPONSE TIME

        try:
            driver.get(base_dados.iloc[i][8])

            sleep(15)

            element = driver.find_element_by_xpath('//*[@id="rootPageContentElementId"]/div/div[2]/div[1]/div[4]/div[2]/div/div[1]/div[1]/div/div/div/div/div//table')
            html_content = element.get_attribute('outerHTML')

            # limpando dados
            soup = BeautifulSoup(html_content, 'html.parser')
            table = soup.find(name='table')

            # criando base de dados
            base_dyna = pd.read_html(str(table))[0]

            if len(base_dyna) > 2:
                response1 = soma_valores_responseTime(base_dyna)
                base_dados['LINK RESPONSE TIME CB'][i] = response1
                sleep(2)

            else:
                base_dados['LINK RESPONSE TIME CB'][i] = base_dyna['Median response time'][0]
                sleep(2)
            

        except:
            base_dados['LINK RESPONSE TIME CB'][i] = 0
            sleep(2)

        


        try:
            driver.get(base_dados.iloc[i][9])

            sleep(15)

            element = driver.find_element_by_xpath('//*[@id="rootPageContentElementId"]/div/div[2]/div[1]/div[4]/div[2]/div/div[1]/div[1]/div/div/div/div/div//table')
            html_content = element.get_attribute('outerHTML')

            # limpando dados
            soup = BeautifulSoup(html_content, 'html.parser')
            table = soup.find(name='table')

            # criando base de dados
            base_dyna = pd.read_html(str(table))[0]

            if len(base_dyna) > 2:
                response2 = soma_valores_responseTime(base_dyna)
                base_dados['LINK RESPONSE TIME EX'][i] = response2
                sleep(2)

            else:
                base_dados['LINK RESPONSE TIME EX'][i] = base_dyna['Median response time'][0]
                sleep(2)

        
        except:
            base_dados['LINK RESPONSE TIME EX'][i] = 0
            sleep(2)




        try:
            driver.get(base_dados.iloc[i][10])

            sleep(15)

            element = driver.find_element_by_xpath('//*[@id="rootPageContentElementId"]/div/div[2]/div[1]/div[4]/div[2]/div/div[1]/div[1]/div/div/div/div/div//table')
            html_content = element.get_attribute('outerHTML')

            # limpando dados
            soup = BeautifulSoup(html_content, 'html.parser')
            table = soup.find(name='table')

            # criando base de dados
            base_dyna = pd.read_html(str(table))[0]

            if len(base_dyna) > 2:
                response3 = soma_valores_responseTime(base_dyna)
                base_dados['LINK RESPONSE TIME PF'][i] = response3
                sleep(2)

            else:
                base_dados['LINK RESPONSE TIME PF'][i] = base_dyna['Median response time'][0]
                sleep(2)

            

        except:
            base_dados['LINK RESPONSE TIME PF'][i] = 0
            sleep(2)




            # FAILURE RATE

        try:
            driver.get(base_dados.iloc[i][11])

            sleep(15)

            element = driver.find_element_by_xpath('//*[@id="rootPageContentElementId"]/div/div[2]/div[1]/div[4]/div[2]/div/div[1]/div[1]/div/div/div/div/div//table')
            html_content = element.get_attribute('outerHTML')

            # limpando dados
            soup = BeautifulSoup(html_content, 'html.parser')
            table = soup.find(name='table')

            # criando base de dados
            base_dyna = pd.read_html(str(table))[0]

            if len(base_dyna) > 2:
                erros4 = soma_valores_taxaErros(base_dyna)
                base_dados['LINK FAILURE RATE CB'][i] = erros4
                sleep(2)

            if len(base_dyna) < 3:
                base_dados['LINK FAILURE RATE CB'][i] = base_dyna['Failure rate'][0]
                sleep(2)


        except:
            base_dados['LINK FAILURE RATE CB'][i] = 0
            sleep(2)

    


        try:
            driver.get(base_dados.iloc[i][12])

            sleep(15)

            element = driver.find_element_by_xpath('//*[@id="rootPageContentElementId"]/div/div[2]/div[1]/div[4]/div[2]/div/div[1]/div[1]/div/div/div/div/div//table')
            html_content = element.get_attribute('outerHTML')

            # limpando dados
            soup = BeautifulSoup(html_content, 'html.parser')
            table = soup.find(name='table')

            # criando base de dados
            base_dyna = pd.read_html(str(table))[0]

            if len(base_dyna) > 2:
                erros5 = soma_valores_responseTime(base_dyna)
                base_dados['LINK FAILURE RATE EX'][i] = erros5
                sleep(2)

            if len(base_dyna) < 3:
                base_dados['LINK FAILURE RATE EX'][i] = base_dyna['Failure rate'][0]
                sleep(2)

        
        except:
            base_dados['LINK FAILURE RATE EX'][i] = 0
            sleep(2)

        


        try:    
            driver.get(base_dados.iloc[i][13])

            sleep(15)

            element = driver.find_element_by_xpath('//*[@id="rootPageContentElementId"]/div/div[2]/div[1]/div[4]/div[2]/div/div[1]/div[1]/div/div/div/div/div//table')
            html_content = element.get_attribute('outerHTML')

            # limpando dados
            soup = BeautifulSoup(html_content, 'html.parser')
            table = soup.find(name='table')

            # criando base de dados
            base_dyna = pd.read_html(str(table))[0]

            if len(base_dyna) > 2:
                erros6 = soma_valores_responseTime(base_dyna)
                base_dados['LINK FAILURE RATE PF'][i] = erros6
                sleep(2)

            if len(base_dyna) < 3:
                base_dados['LINK FAILURE RATE PF'][i] = base_dyna['Failure rate'][0]
                sleep(2)

        except:
            base_dados['LINK FAILURE RATE PF'][i] = 0
            sleep(2)




        # CPU

        try:
            driver.get(base_dados.iloc[i][14])

            sleep(15)

            element = driver.find_element_by_xpath('//*[@id="rootPageContentElementId"]/div/div[2]/div[1]/div[4]/div[2]/div/div[1]/div[1]/div/div/div/div/div//table')
            html_content = element.get_attribute('outerHTML')

            # limpando dados
            soup = BeautifulSoup(html_content, 'html.parser')
            table = soup.find(name='table')

            # criando base de dados
            base_dyna = pd.read_html(str(table))[0]


            if len(base_dyna) > 2:
                cpu7 = soma_valores_CPU(base_dyna)
                base_dados['LINK CPU CB'][i] = cpu7
                sleep(2)

            else:
                base_dados['LINK CPU CB'][i] = base_dyna['CPU'][0]
                sleep(2)



        except:
            base_dados['LINK CPU CB'][i] = 0
            sleep(2)

        
        try:
            driver.get(base_dados.iloc[i][15])

            sleep(15)

            element = driver.find_element_by_xpath('//*[@id="rootPageContentElementId"]/div/div[2]/div[1]/div[4]/div[2]/div/div[1]/div[1]/div/div/div/div/div//table')
            html_content = element.get_attribute('outerHTML')

            # limpando dados
            soup = BeautifulSoup(html_content, 'html.parser')
            table = soup.find(name='table')

            # criando base de dados
            base_dyna = pd.read_html(str(table))[0]

            if len(base_dyna) > 2:
                cpu8 = soma_valores_CPU(base_dyna)
                base_dados['LINK CPU EX'][i] = cpu8
                sleep(2)

            else:
                base_dados['LINK CPU EX'][i] = base_dyna['CPU'][0]
                sleep(2)

            

        except:
            base_dados['LINK CPU EX'][i] = 0
            sleep(2)


        try:
            driver.get(base_dados.iloc[i][16])

            sleep(15)

            element = driver.find_element_by_xpath('//*[@id="rootPageContentElementId"]/div/div[2]/div[1]/div[4]/div[2]/div/div[1]/div[1]/div/div/div/div/div//table')
            html_content = element.get_attribute('outerHTML')

            # limpando dados
            soup = BeautifulSoup(html_content, 'html.parser')
            table = soup.find(name='table')

            # criando base de dados
            base_dyna = pd.read_html(str(table))[0]

            if len(base_dyna) > 2:
                cpu9 = soma_valores_CPU(base_dyna)
                base_dados['LINK CPU PF'][i] = cpu9
                sleep(2)

            else:
                base_dados['LINK CPU PF'][i] = base_dyna['CPU'][0]
                sleep(2)

        except:
            base_dados['LINK CPU PF'][i] = 0
            sleep(2)



    

        # REQUESTS

        try:
            driver.get(base_dados.iloc[i][17])

            sleep(15)

            element = driver.find_element_by_xpath('//*[@id="rootPageContentElementId"]/div/div[2]/div[1]/div[4]/div[2]/div/div[1]/div[1]/div/div/div/div/div//table')
            html_content = element.get_attribute('outerHTML')

            # limpando dados
            soup = BeautifulSoup(html_content, 'html.parser')
            table = soup.find(name='table')

            # criando base de dados
            base_dyna = pd.read_html(str(table))[0]

            if len(base_dyna) > 2:
                request10 = soma_valores_Requests(base_dyna)
                base_dados['LINK REQUESTS CB'][i] = request10
                sleep(2)

            else:
                base_dados['LINK REQUESTS CB'][i] = base_dyna['Processed requests'][0]
                sleep(2)


        except:
            base_dados['LINK REQUESTS CB'][i] = 0
            sleep(2)



        try:
            driver.get(base_dados.iloc[i][18])

            sleep(15)

            element = driver.find_element_by_xpath('//*[@id="rootPageContentElementId"]/div/div[2]/div[1]/div[4]/div[2]/div/div[1]/div[1]/div/div/div/div/div//table')
            html_content = element.get_attribute('outerHTML')

            # limpando dados
            soup = BeautifulSoup(html_content, 'html.parser')
            table = soup.find(name='table')

            # criando base de dados
            base_dyna = pd.read_html(str(table))[0]

            if len(base_dyna) > 2:
                request11 = soma_valores_Requests(base_dyna)
                base_dados['LINK REQUESTS EX'][i] = request11
                sleep(2)

            else:
                base_dados['LINK REQUESTS EX'][i] = base_dyna['Processed requests'][0]
                sleep(2)


        except:
            base_dados['LINK REQUESTS EX'][i] = 0
            sleep(2)



        try:
            driver.get(base_dados.iloc[i][19])

            sleep(15)

            element = driver.find_element_by_xpath('//*[@id="rootPageContentElementId"]/div/div[2]/div[1]/div[4]/div[2]/div/div[1]/div[1]/div/div/div/div/div//table')
            html_content = element.get_attribute('outerHTML')

            # limpando dados
            soup = BeautifulSoup(html_content, 'html.parser')
            table = soup.find(name='table')

            # criando base de dados
            base_dyna = pd.read_html(str(table))[0]

            if len(base_dyna) > 2:
                request12 = soma_valores_Requests(base_dyna)
                base_dados['LINK REQUESTS PF'][i] = request12
                sleep(2)

            else:
                base_dados['LINK REQUESTS PF'][i] = base_dyna['Processed requests'][0]
                sleep(2)

        

        except:
            base_dados['LINK REQUESTS PF'][i] = 0
            sleep(2)

else:
    pass

C:\Users\Andre\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\Andre\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Andre\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Andre\AppData\Roaming\P

In [50]:
# Criando DF da bateria 1 
bateria_1 = base_dados.copy()

In [51]:
# LIMPEZA DE DADOS NA BATERIA 1

bateria_1['LINK RESPONSE TIME CB'] = limpeza_responseTime(bateria_1['LINK RESPONSE TIME CB'])

bateria_1['LINK RESPONSE TIME EX'] = limpeza_responseTime(bateria_1['LINK RESPONSE TIME EX'])

bateria_1['LINK RESPONSE TIME PF'] = limpeza_responseTime(bateria_1['LINK RESPONSE TIME PF'])

############################################################################################################

bateria_1['LINK FAILURE RATE CB'] = limpeza_taxaErros(bateria_1['LINK FAILURE RATE CB'])

bateria_1['LINK FAILURE RATE EX'] = limpeza_taxaErros(bateria_1['LINK FAILURE RATE EX'])

bateria_1['LINK FAILURE RATE PF'] = limpeza_taxaErros(bateria_1['LINK FAILURE RATE PF'])

############################################################################################################

bateria_1['LINK CPU CB'] = limpeza_CPU(bateria_1['LINK CPU CB'])

bateria_1['LINK CPU EX'] = limpeza_CPU(bateria_1['LINK CPU EX'])

bateria_1['LINK CPU PF'] = limpeza_CPU(bateria_1['LINK CPU PF'])

############################################################################################################

bateria_1['LINK REQUESTS CB'] = limpeza_Requests(bateria_1['LINK REQUESTS CB'])

bateria_1['LINK REQUESTS EX'] = limpeza_Requests(bateria_1['LINK REQUESTS EX'])

bateria_1['LINK REQUESTS PF'] = limpeza_Requests(bateria_1['LINK REQUESTS PF'])

C:\Users\Andre\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Andre\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Andre\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [52]:
bateria_1 = bateria_1.rename(columns={'LINK RESPONSE TIME CB': 'Resultado CB Response Time ms', 'LINK RESPONSE TIME EX': 'Resultado EX Response Time ms', 'LINK RESPONSE TIME PF': 'Resultado PF Response Time ms', 'LINK FAILURE RATE CB': 'Resultado CB Taxa de Erros %', 'LINK FAILURE RATE EX': 'Resultado EX Taxa de Erros %', 'LINK FAILURE RATE PF':'Resultado PF Taxa de Erros %', 'LINK CPU CB': 'Resultado CB Consumo CPU %', 'LINK CPU EX': 'Resultado EX Consumo CPU %', 'LINK CPU PF': 'Resultado PF Consumo CPU %', 'LINK REQUESTS CB': 'Resultado CB Throughput Reqs/min', 'LINK REQUESTS EX': 'Resultado EX Throughput Reqs/min', 'LINK REQUESTS PF':'Resultado PF Throughput Reqs/min'})

In [53]:
# FORMATANDO NÚMEROS PARA MANTIMENTO DE UM PADRÃO

for i in range(len(bateria_1)):
    try:
        bateria_1['Resultado CB Response Time ms'][i] = float(bateria_1['Resultado CB Response Time ms'][i])

        bateria_1['Resultado CB Response Time ms'][i] = float(f'{bateria_1["Resultado CB Response Time ms"][i]:.1f}')

    except:
        pass

    
    try:
        bateria_1['Resultado EX Response Time ms'][i] = float(bateria_1['Resultado EX Response Time ms'][i])

        bateria_1['Resultado EX Response Time ms'][i] = float(f'{bateria_1["Resultado EX Response Time ms"][i]:.1f}')

    except:
        pass


    try:
        bateria_1['Resultado PF Response Time ms'][i] = float(bateria_1['Resultado PF Response Time ms'][i])

        bateria_1['Resultado PF Response Time ms'][i] = float(f'{bateria_1["Resultado PF Response Time ms"][i]:.1f}')

    except:
        pass


    try:
        bateria_1['Resultado CB Taxa de Erros %'][i] = float(bateria_1['Resultado CB Taxa de Erros %'][i])

        bateria_1['Resultado CB Taxa de Erros %'][i] = float(f'{bateria_1["Resultado CB Taxa de Erros %"][i]:.1f}')

    except:
        pass


    try:
        bateria_1['Resultado EX Taxa de Erros %'][i] = float(bateria_1['Resultado EX Taxa de Erros %'][i])

        bateria_1['Resultado EX Taxa de Erros %'][i] = float(f'{bateria_1["Resultado EX Taxa de Erros %"][i]:.1f}')

    except:
        pass


    try:
        bateria_1['Resultado PF Taxa de Erros %'][i] = float(bateria_1['Resultado PF Taxa de Erros %'][i])

        bateria_1['Resultado PF Taxa de Erros %'][i] = float(f'{bateria_1["Resultado PF Taxa de Erros %"][i]:.1f}')

    except:
        pass



    try:
        bateria_1['Resultado CB Consumo CPU %'][i] = float(bateria_1['Resultado CB Consumo CPU %'][i])

        bateria_1['Resultado CB Consumo CPU %'][i] = float(f'{bateria_1["Resultado CB Consumo CPU %"][i]:.1f}')

    except:
        pass


    try:
        bateria_1['Resultado EX Consumo CPU %'][i] = float(bateria_1['Resultado EX Consumo CPU %'][i])

        bateria_1['Resultado EX Consumo CPU %'][i] = float(f'{bateria_1["Resultado EX Consumo CPU %"][i]:.1f}')

    except:
        pass


    try:
        bateria_1['Resultado PF Consumo CPU %'][i] = float(bateria_1['Resultado PF Consumo CPU %'][i])

        bateria_1['Resultado PF Consumo CPU %'][i] = float(f'{bateria_1["Resultado PF Consumo CPU %"][i]:.1f}')

    except:
        pass



    try:
        bateria_1['Resultado CB Throughput Reqs/min'][i] = float(bateria_1['Resultado CB Throughput Reqs/min'][i])

        bateria_1['Resultado CB Throughput Reqs/min'][i] = float(f'{bateria_1["Resultado CB Throughput Reqs/min"][i]:.1f}')

    except:
        pass



    try:
        bateria_1['Resultado EX Throughput Reqs/min'][i] = float(bateria_1['Resultado EX Throughput Reqs/min'][i])

        bateria_1['Resultado EX Throughput Reqs/min'][i] = float(f'{bateria_1["Resultado EX Throughput Reqs/min"][i]:.1f}')

    except:
        pass


    try:
        bateria_1['Resultado PF Throughput Reqs/min'][i] = float(bateria_1['Resultado PF Throughput Reqs/min'][i])

        bateria_1['Resultado PF Throughput Reqs/min'][i] = float(f'{bateria_1["Resultado PF Throughput Reqs/min"][i]:.1f}')

    except:
        pass



C:\Users\Andre\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\Andre\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\Andre\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Andre\AppData\Roa

# 3.0 Segunda Bateria do Teste

## 3.1 Zerando Base de Dados

In [55]:
base_dados = pd.read_excel(r'C:\Users\Andre\Documents\Via\Base_Dados_Manuais\ParceriasB2B_BaseDados_Automacao.xlsx')

## 3.2 Imputando Data e Hora da Segunda Bateria do Teste

In [56]:
# ALTERANDO O LINK COM A DATA E HORA DA SEGUNDA BATERIA

for i in range(len(base_dados)):
    base_dados['LINK RESPONSE TIME CB'].iloc[i] = str(base_dados['LINK RESPONSE TIME CB'].iloc[i]).replace(data_anterior_2, data_atual_2)
    base_dados['LINK RESPONSE TIME EX'].iloc[i] = str(base_dados['LINK RESPONSE TIME EX'].iloc[i]).replace(data_anterior_2, data_atual_2)
    base_dados['LINK RESPONSE TIME PF'].iloc[i] = str(base_dados['LINK RESPONSE TIME PF'].iloc[i]).replace(data_anterior_2, data_atual_2)


    base_dados['LINK FAILURE RATE CB'].iloc[i] = str(base_dados['LINK FAILURE RATE CB'].iloc[i]).replace(data_anterior_2, data_atual_2)
    base_dados['LINK FAILURE RATE EX'].iloc[i] = str(base_dados['LINK FAILURE RATE EX'].iloc[i]).replace(data_anterior_2, data_atual_2)
    base_dados['LINK FAILURE RATE PF'].iloc[i] = str(base_dados['LINK FAILURE RATE PF'].iloc[i]).replace(data_anterior_2, data_atual_2)

    base_dados['LINK CPU CB'].iloc[i] = str(base_dados['LINK CPU CB'].iloc[i]).replace(data_anterior_2, data_atual_2)
    base_dados['LINK CPU EX'].iloc[i] = str(base_dados['LINK CPU EX'].iloc[i]).replace(data_anterior_2, data_atual_2)
    base_dados['LINK CPU PF'].iloc[i] = str(base_dados['LINK CPU PF'].iloc[i]).replace(data_anterior_2, data_atual_2)

    base_dados['LINK REQUESTS CB'].iloc[i] = str(base_dados['LINK REQUESTS CB'].iloc[i]).replace(data_anterior_2, data_atual_2)
    base_dados['LINK REQUESTS EX'].iloc[i] = str(base_dados['LINK REQUESTS EX'].iloc[i]).replace(data_anterior_2, data_atual_2)
    base_dados['LINK REQUESTS PF'].iloc[i] = str(base_dados['LINK REQUESTS PF'].iloc[i]).replace(data_anterior_2, data_atual_2)

C:\Users\Andre\AppData\Roaming\Python\Python37\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


## 3.4 Rodando a Extração dos Dados da Segunda Bateria do Teste

In [57]:
# BATERIA 2

# Extraindo dados de RESPONSE TIME, FAILURE RATE, CPU e REQUESTS diretamente do dynatrace 

if 2 <= int(num_bateria): 

    for i in range(len(base_dados)):

        base_dados['BATERIA'][i] = 2
        base_dados['DATA'][i] = data

        # RESPONSE TIME

        try:
            driver.get(base_dados.iloc[i][8])

            sleep(15)

            element = driver.find_element_by_xpath('//*[@id="rootPageContentElementId"]/div/div[2]/div[1]/div[4]/div[2]/div/div[1]/div[1]/div/div/div/div/div//table')
            html_content = element.get_attribute('outerHTML')

            # limpando dados
            soup = BeautifulSoup(html_content, 'html.parser')
            table = soup.find(name='table')

            # criando base de dados
            base_dyna = pd.read_html(str(table))[0]

            if len(base_dyna) > 2:
                response1 = soma_valores_responseTime(base_dyna)
                base_dados['LINK RESPONSE TIME CB'][i] = response1
                sleep(2)

            else:
                base_dados['LINK RESPONSE TIME CB'][i] = base_dyna['Median response time'][0]
                sleep(2)
            

        except:
            base_dados['LINK RESPONSE TIME CB'][i] = 0
            sleep(2)

        


        try:
            driver.get(base_dados.iloc[i][9])

            sleep(15)

            element = driver.find_element_by_xpath('//*[@id="rootPageContentElementId"]/div/div[2]/div[1]/div[4]/div[2]/div/div[1]/div[1]/div/div/div/div/div//table')
            html_content = element.get_attribute('outerHTML')

            # limpando dados
            soup = BeautifulSoup(html_content, 'html.parser')
            table = soup.find(name='table')

            # criando base de dados
            base_dyna = pd.read_html(str(table))[0]

            if len(base_dyna) > 2:
                response2 = soma_valores_responseTime(base_dyna)
                base_dados['LINK RESPONSE TIME EX'][i] = response2
                sleep(2)

            else:
                base_dados['LINK RESPONSE TIME EX'][i] = base_dyna['Median response time'][0]
                sleep(2)

        
        except:
            base_dados['LINK RESPONSE TIME EX'][i] = 0
            sleep(2)




        try:
            driver.get(base_dados.iloc[i][10])

            sleep(15)

            element = driver.find_element_by_xpath('//*[@id="rootPageContentElementId"]/div/div[2]/div[1]/div[4]/div[2]/div/div[1]/div[1]/div/div/div/div/div//table')
            html_content = element.get_attribute('outerHTML')

            # limpando dados
            soup = BeautifulSoup(html_content, 'html.parser')
            table = soup.find(name='table')

            # criando base de dados
            base_dyna = pd.read_html(str(table))[0]

            if len(base_dyna) > 2:
                response3 = soma_valores_responseTime(base_dyna)
                base_dados['LINK RESPONSE TIME PF'][i] = response3
                sleep(2)

            else:
                base_dados['LINK RESPONSE TIME PF'][i] = base_dyna['Median response time'][0]
                sleep(2)

            

        except:
            base_dados['LINK RESPONSE TIME PF'][i] = 0
            sleep(2)




            # FAILURE RATE

        try:
            driver.get(base_dados.iloc[i][11])

            sleep(15)

            element = driver.find_element_by_xpath('//*[@id="rootPageContentElementId"]/div/div[2]/div[1]/div[4]/div[2]/div/div[1]/div[1]/div/div/div/div/div//table')
            html_content = element.get_attribute('outerHTML')

            # limpando dados
            soup = BeautifulSoup(html_content, 'html.parser')
            table = soup.find(name='table')

            # criando base de dados
            base_dyna = pd.read_html(str(table))[0]

            if len(base_dyna) > 2:
                erros4 = soma_valores_taxaErros(base_dyna)
                base_dados['LINK FAILURE RATE CB'][i] = erros4
                sleep(2)

            if len(base_dyna) < 3:
                base_dados['LINK FAILURE RATE CB'][i] = base_dyna['Failure rate'][0]
                sleep(2)


        except:
            base_dados['LINK FAILURE RATE CB'][i] = 0
            sleep(2)

    


        try:
            driver.get(base_dados.iloc[i][12])

            sleep(15)

            element = driver.find_element_by_xpath('//*[@id="rootPageContentElementId"]/div/div[2]/div[1]/div[4]/div[2]/div/div[1]/div[1]/div/div/div/div/div//table')
            html_content = element.get_attribute('outerHTML')

            # limpando dados
            soup = BeautifulSoup(html_content, 'html.parser')
            table = soup.find(name='table')

            # criando base de dados
            base_dyna = pd.read_html(str(table))[0]

            if len(base_dyna) > 2:
                erros5 = soma_valores_responseTime(base_dyna)
                base_dados['LINK FAILURE RATE EX'][i] = erros5
                sleep(2)

            if len(base_dyna) < 3:
                base_dados['LINK FAILURE RATE EX'][i] = base_dyna['Failure rate'][0]
                sleep(2)

        
        except:
            base_dados['LINK FAILURE RATE EX'][i] = 0
            sleep(2)

        


        try:    
            driver.get(base_dados.iloc[i][13])

            sleep(15)

            element = driver.find_element_by_xpath('//*[@id="rootPageContentElementId"]/div/div[2]/div[1]/div[4]/div[2]/div/div[1]/div[1]/div/div/div/div/div//table')
            html_content = element.get_attribute('outerHTML')

            # limpando dados
            soup = BeautifulSoup(html_content, 'html.parser')
            table = soup.find(name='table')

            # criando base de dados
            base_dyna = pd.read_html(str(table))[0]

            if len(base_dyna) > 2:
                erros6 = soma_valores_responseTime(base_dyna)
                base_dados['LINK FAILURE RATE PF'][i] = erros6
                sleep(2)

            if len(base_dyna) < 3:
                base_dados['LINK FAILURE RATE PF'][i] = base_dyna['Failure rate'][0]
                sleep(2)

        except:
            base_dados['LINK FAILURE RATE PF'][i] = 0
            sleep(2)




        # CPU

        try:
            driver.get(base_dados.iloc[i][14])

            sleep(15)

            element = driver.find_element_by_xpath('//*[@id="rootPageContentElementId"]/div/div[2]/div[1]/div[4]/div[2]/div/div[1]/div[1]/div/div/div/div/div//table')
            html_content = element.get_attribute('outerHTML')

            # limpando dados
            soup = BeautifulSoup(html_content, 'html.parser')
            table = soup.find(name='table')

            # criando base de dados
            base_dyna = pd.read_html(str(table))[0]


            if len(base_dyna) > 2:
                cpu7 = soma_valores_CPU(base_dyna)
                base_dados['LINK CPU CB'][i] = cpu7
                sleep(2)

            else:
                base_dados['LINK CPU CB'][i] = base_dyna['CPU'][0]
                sleep(2)



        except:
            base_dados['LINK CPU CB'][i] = 0
            sleep(2)

        
        try:
            driver.get(base_dados.iloc[i][15])

            sleep(15)

            element = driver.find_element_by_xpath('//*[@id="rootPageContentElementId"]/div/div[2]/div[1]/div[4]/div[2]/div/div[1]/div[1]/div/div/div/div/div//table')
            html_content = element.get_attribute('outerHTML')

            # limpando dados
            soup = BeautifulSoup(html_content, 'html.parser')
            table = soup.find(name='table')

            # criando base de dados
            base_dyna = pd.read_html(str(table))[0]

            if len(base_dyna) > 2:
                cpu8 = soma_valores_CPU(base_dyna)
                base_dados['LINK CPU EX'][i] = cpu8
                sleep(2)

            else:
                base_dados['LINK CPU EX'][i] = base_dyna['CPU'][0]
                sleep(2)

            

        except:
            base_dados['LINK CPU EX'][i] = 0
            sleep(2)


        try:
            driver.get(base_dados.iloc[i][16])

            sleep(15)

            element = driver.find_element_by_xpath('//*[@id="rootPageContentElementId"]/div/div[2]/div[1]/div[4]/div[2]/div/div[1]/div[1]/div/div/div/div/div//table')
            html_content = element.get_attribute('outerHTML')

            # limpando dados
            soup = BeautifulSoup(html_content, 'html.parser')
            table = soup.find(name='table')

            # criando base de dados
            base_dyna = pd.read_html(str(table))[0]

            if len(base_dyna) > 2:
                cpu9 = soma_valores_CPU(base_dyna)
                base_dados['LINK CPU PF'][i] = cpu9
                sleep(2)

            else:
                base_dados['LINK CPU PF'][i] = base_dyna['CPU'][0]
                sleep(2)

        except:
            base_dados['LINK CPU PF'][i] = 0
            sleep(2)



    

        # REQUESTS

        try:
            driver.get(base_dados.iloc[i][17])

            sleep(15)

            element = driver.find_element_by_xpath('//*[@id="rootPageContentElementId"]/div/div[2]/div[1]/div[4]/div[2]/div/div[1]/div[1]/div/div/div/div/div//table')
            html_content = element.get_attribute('outerHTML')

            # limpando dados
            soup = BeautifulSoup(html_content, 'html.parser')
            table = soup.find(name='table')

            # criando base de dados
            base_dyna = pd.read_html(str(table))[0]

            if len(base_dyna) > 2:
                request10 = soma_valores_Requests(base_dyna)
                base_dados['LINK REQUESTS CB'][i] = request10
                sleep(2)

            else:
                base_dados['LINK REQUESTS CB'][i] = base_dyna['Processed requests'][0]
                sleep(2)


        except:
            base_dados['LINK REQUESTS CB'][i] = 0
            sleep(2)



        try:
            driver.get(base_dados.iloc[i][18])

            sleep(15)

            element = driver.find_element_by_xpath('//*[@id="rootPageContentElementId"]/div/div[2]/div[1]/div[4]/div[2]/div/div[1]/div[1]/div/div/div/div/div//table')
            html_content = element.get_attribute('outerHTML')

            # limpando dados
            soup = BeautifulSoup(html_content, 'html.parser')
            table = soup.find(name='table')

            # criando base de dados
            base_dyna = pd.read_html(str(table))[0]

            if len(base_dyna) > 2:
                request11 = soma_valores_Requests(base_dyna)
                base_dados['LINK REQUESTS EX'][i] = request11
                sleep(2)

            else:
                base_dados['LINK REQUESTS EX'][i] = base_dyna['Processed requests'][0]
                sleep(2)


        except:
            base_dados['LINK REQUESTS EX'][i] = 0
            sleep(2)



        try:
            driver.get(base_dados.iloc[i][19])

            sleep(15)

            element = driver.find_element_by_xpath('//*[@id="rootPageContentElementId"]/div/div[2]/div[1]/div[4]/div[2]/div/div[1]/div[1]/div/div/div/div/div//table')
            html_content = element.get_attribute('outerHTML')

            # limpando dados
            soup = BeautifulSoup(html_content, 'html.parser')
            table = soup.find(name='table')

            # criando base de dados
            base_dyna = pd.read_html(str(table))[0]

            if len(base_dyna) > 2:
                request12 = soma_valores_Requests(base_dyna)
                base_dados['LINK REQUESTS PF'][i] = request12
                sleep(2)

            else:
                base_dados['LINK REQUESTS PF'][i] = base_dyna['Processed requests'][0]
                sleep(2)

        

        except:
            base_dados['LINK REQUESTS PF'][i] = 0
            sleep(2)

else:
    pass

C:\Users\Andre\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\Andre\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Andre\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Andre\AppData\Roaming\P

In [58]:
bateria_2 = base_dados.copy()

In [59]:
# LIMPEZA DE DADOS NA BATERIA 2

bateria_2['LINK RESPONSE TIME CB'] = limpeza_responseTime(bateria_2['LINK RESPONSE TIME CB'])

bateria_2['LINK RESPONSE TIME EX'] = limpeza_responseTime(bateria_2['LINK RESPONSE TIME EX'])

bateria_2['LINK RESPONSE TIME PF'] = limpeza_responseTime(bateria_2['LINK RESPONSE TIME PF'])

############################################################################################################

bateria_2['LINK FAILURE RATE CB'] = limpeza_taxaErros(bateria_2['LINK FAILURE RATE CB'])

bateria_2['LINK FAILURE RATE EX'] = limpeza_taxaErros(bateria_2['LINK FAILURE RATE EX'])

bateria_2['LINK FAILURE RATE PF'] = limpeza_taxaErros(bateria_2['LINK FAILURE RATE PF'])

############################################################################################################

bateria_2['LINK CPU CB'] = limpeza_CPU(bateria_2['LINK CPU CB'])

bateria_2['LINK CPU EX'] = limpeza_CPU(bateria_2['LINK CPU EX'])

bateria_2['LINK CPU PF'] = limpeza_CPU(bateria_2['LINK CPU PF'])

############################################################################################################

bateria_2['LINK REQUESTS CB'] = limpeza_Requests(bateria_2['LINK REQUESTS CB'])

bateria_2['LINK REQUESTS EX'] = limpeza_Requests(bateria_2['LINK REQUESTS EX'])

bateria_2['LINK REQUESTS PF'] = limpeza_Requests(bateria_2['LINK REQUESTS PF'])

C:\Users\Andre\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Andre\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Andre\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [60]:
bateria_2 = bateria_2.rename(columns={'LINK RESPONSE TIME CB': 'Resultado CB Response Time ms', 'LINK RESPONSE TIME EX': 'Resultado EX Response Time ms', 'LINK RESPONSE TIME PF': 'Resultado PF Response Time ms', 'LINK FAILURE RATE CB': 'Resultado CB Taxa de Erros %', 'LINK FAILURE RATE EX': 'Resultado EX Taxa de Erros %', 'LINK FAILURE RATE PF':'Resultado PF Taxa de Erros %', 'LINK CPU CB': 'Resultado CB Consumo CPU %', 'LINK CPU EX': 'Resultado EX Consumo CPU %', 'LINK CPU PF': 'Resultado PF Consumo CPU %', 'LINK REQUESTS CB': 'Resultado CB Throughput Reqs/min', 'LINK REQUESTS EX': 'Resultado EX Throughput Reqs/min', 'LINK REQUESTS PF':'Resultado PF Throughput Reqs/min'})

In [61]:
# FORMATANDO NÚMEROS PARA MANTIMENTO DE UM PADRÃO

for i in range(len(bateria_2)):
    try:
        bateria_2['Resultado CB Response Time ms'][i] = float(bateria_2['Resultado CB Response Time ms'][i])

        bateria_2['Resultado CB Response Time ms'][i] = float(f'{bateria_2["Resultado CB Response Time ms"][i]:.1f}')

    except:
        pass

    
    try:
        bateria_2['Resultado EX Response Time ms'][i] = float(bateria_2['Resultado EX Response Time ms'][i])

        bateria_2['Resultado EX Response Time ms'][i] = float(f'{bateria_2["Resultado EX Response Time ms"][i]:.1f}')

    except:
        pass


    try:
        bateria_2['Resultado PF Response Time ms'][i] = float(bateria_2['Resultado PF Response Time ms'][i])

        bateria_2['Resultado PF Response Time ms'][i] = float(f'{bateria_2["Resultado PF Response Time ms"][i]:.1f}')

    except:
        pass


    try:
        bateria_2['Resultado CB Taxa de Erros %'][i] = float(bateria_2['Resultado CB Taxa de Erros %'][i])

        bateria_2['Resultado CB Taxa de Erros %'][i] = float(f'{bateria_2["Resultado CB Taxa de Erros %"][i]:.1f}')

    except:
        pass


    try:
        bateria_2['Resultado EX Taxa de Erros %'][i] = float(bateria_2['Resultado EX Taxa de Erros %'][i])

        bateria_2['Resultado EX Taxa de Erros %'][i] = float(f'{bateria_2["Resultado EX Taxa de Erros %"][i]:.1f}')

    except:
        pass


    try:
        bateria_2['Resultado PF Taxa de Erros %'][i] = float(bateria_2['Resultado PF Taxa de Erros %'][i])

        bateria_2['Resultado PF Taxa de Erros %'][i] = float(f'{bateria_2["Resultado PF Taxa de Erros %"][i]:.1f}')

    except:
        pass



    try:
        bateria_2['Resultado CB Consumo CPU %'][i] = float(bateria_2['Resultado CB Consumo CPU %'][i])

        bateria_2['Resultado CB Consumo CPU %'][i] = float(f'{bateria_2["Resultado CB Consumo CPU %"][i]:.1f}')

    except:
        pass


    try:
        bateria_2['Resultado EX Consumo CPU %'][i] = float(bateria_2['Resultado EX Consumo CPU %'][i])

        bateria_2['Resultado EX Consumo CPU %'][i] = float(f'{bateria_2["Resultado EX Consumo CPU %"][i]:.1f}')

    except:
        pass


    try:
        bateria_2['Resultado PF Consumo CPU %'][i] = float(bateria_2['Resultado PF Consumo CPU %'][i])

        bateria_2['Resultado PF Consumo CPU %'][i] = float(f'{bateria_2["Resultado PF Consumo CPU %"][i]:.1f}')

    except:
        pass



    try:
        bateria_2['Resultado CB Throughput Reqs/min'][i] = float(bateria_2['Resultado CB Throughput Reqs/min'][i])

        bateria_2['Resultado CB Throughput Reqs/min'][i] = float(f'{bateria_2["Resultado CB Throughput Reqs/min"][i]:.1f}')

    except:
        pass



    try:
        bateria_2['Resultado EX Throughput Reqs/min'][i] = float(bateria_2['Resultado EX Throughput Reqs/min'][i])

        bateria_2['Resultado EX Throughput Reqs/min'][i] = float(f'{bateria_2["Resultado EX Throughput Reqs/min"][i]:.1f}')

    except:
        pass


    try:
        bateria_2['Resultado PF Throughput Reqs/min'][i] = float(bateria_2['Resultado PF Throughput Reqs/min'][i])

        bateria_2['Resultado PF Throughput Reqs/min'][i] = float(f'{bateria_2["Resultado PF Throughput Reqs/min"][i]:.1f}')

    except:
        pass



C:\Users\Andre\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\Andre\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\Andre\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Andre\AppData\Roa

# 4.0 Terceira Bateria do Teste

## 4.1 Zerando Base de Dados

In [ ]:
base_dados = pd.read_excel(r'C:\Users\Andre\Documents\Via\Base_Dados_Manuais\CompraSelfService_BaseDados_Automacao.xlsx')

## 4.2 Imputando Dados de Data e Hora da Terceira Bateria

In [ ]:
# ALTERANDO O LINK COM A DATA E HORA DA TERCEIRA BATERIA
for i in range(len(base_dados)):
    base_dados['LINK RESPONSE TIME CB'].iloc[i] = str(base_dados['LINK RESPONSE TIME CB'].iloc[i].replace(data_anterior_3, data_atual_3))
    base_dados['LINK RESPONSE TIME EX'].iloc[i] = str(base_dados['LINK RESPONSE TIME EX'].iloc[i].replace(data_anterior_3, data_atual_3))
    base_dados['LINK RESPONSE TIME PF'].iloc[i] = str(base_dados['LINK RESPONSE TIME PF'].iloc[i].replace(data_anterior_3, data_atual_3))


    base_dados['LINK FAILURE RATE CB'].iloc[i] = str(base_dados['LINK FAILURE RATE CB'].iloc[i].replace(data_anterior_3, data_atual_3))
    base_dados['LINK FAILURE RATE EX'].iloc[i] = str(base_dados['LINK FAILURE RATE EX'].iloc[i].replace(data_anterior_3, data_atual_3))
    base_dados['LINK FAILURE RATE PF'].iloc[i] = str(base_dados['LINK FAILURE RATE PF'].iloc[i].replace(data_anterior_1, data_atual_3))

    base_dados['LINK CPU CB'].iloc[i] = str(base_dados['LINK CPU CB'].iloc[i].replace(data_anterior_3, data_atual_3))
    base_dados['LINK CPU EX'].iloc[i] = str(base_dados['LINK CPU EX'].iloc[i].replace(data_anterior_3, data_atual_3))
    base_dados['LINK CPU PF'].iloc[i] = str(base_dados['LINK CPU PF'].iloc[i].replace(data_anterior_3, data_atual_3))

    base_dados['LINK REQUESTS CB'].iloc[i] = str(base_dados['LINK REQUESTS CB'].iloc[i].replace(data_anterior_3, data_atual_3))
    base_dados['LINK REQUESTS EX'].iloc[i] = str(base_dados['LINK REQUESTS EX'].iloc[i].replace(data_anterior_3, data_atual_3))
    base_dados['LINK REQUESTS PF'].iloc[i] = str(base_dados['LINK REQUESTS PF'].iloc[i].replace(data_anterior_3, data_atual_3))

## 4.3 Rodando Extração de Dados da Terceira Bateria

In [ ]:
# BATERIA 3

# Extraindo dados de RESPONSE TIME, FAILURE RATE, CPU e REQUESTS diretamente do dynatrace 

if 3 <= int(num_bateria): 

    for i in range(len(base_dados)):

        base_dados['BATERIA'][i] = 3
        base_dados['DATA'][i] = data

        # RESPONSE TIME

        try:
            driver.get(base_dados.iloc[i][8])

            sleep(15)

            element = driver.find_element_by_xpath('//*[@id="rootPageContentElementId"]/div/div[2]/div[1]/div[4]/div[2]/div/div[1]/div[1]/div/div/div/div/div//table')
            html_content = element.get_attribute('outerHTML')

            # limpando dados
            soup = BeautifulSoup(html_content, 'html.parser')
            table = soup.find(name='table')

            # criando base de dados
            base_dyna = pd.read_html(str(table))[0]

            if len(base_dyna) > 2:
                response1 = soma_valores_responseTime(base_dyna)
                base_dados['LINK RESPONSE TIME CB'][i] = response1
                sleep(2)

            else:
                base_dados['LINK RESPONSE TIME CB'][i] = base_dyna['Median response time'][0]
                sleep(2)
            

        except:
            base_dados['LINK RESPONSE TIME CB'][i] = 0
            sleep(2)

        


        try:
            driver.get(base_dados.iloc[i][9])

            sleep(15)

            element = driver.find_element_by_xpath('//*[@id="rootPageContentElementId"]/div/div[2]/div[1]/div[4]/div[2]/div/div[1]/div[1]/div/div/div/div/div//table')
            html_content = element.get_attribute('outerHTML')

            # limpando dados
            soup = BeautifulSoup(html_content, 'html.parser')
            table = soup.find(name='table')

            # criando base de dados
            base_dyna = pd.read_html(str(table))[0]

            if len(base_dyna) > 2:
                response2 = soma_valores_responseTime(base_dyna)
                base_dados['LINK RESPONSE TIME EX'][i] = response2
                sleep(2)

            else:
                base_dados['LINK RESPONSE TIME EX'][i] = base_dyna['Median response time'][0]
                sleep(2)

        
        except:
            base_dados['LINK RESPONSE TIME EX'][i] = 0
            sleep(2)




        try:
            driver.get(base_dados.iloc[i][10])

            sleep(15)

            element = driver.find_element_by_xpath('//*[@id="rootPageContentElementId"]/div/div[2]/div[1]/div[4]/div[2]/div/div[1]/div[1]/div/div/div/div/div//table')
            html_content = element.get_attribute('outerHTML')

            # limpando dados
            soup = BeautifulSoup(html_content, 'html.parser')
            table = soup.find(name='table')

            # criando base de dados
            base_dyna = pd.read_html(str(table))[0]

            if len(base_dyna) > 2:
                response3 = soma_valores_responseTime(base_dyna)
                base_dados['LINK RESPONSE TIME PF'][i] = response3
                sleep(2)

            else:
                base_dados['LINK RESPONSE TIME PF'][i] = base_dyna['Median response time'][0]
                sleep(2)

            

        except:
            base_dados['LINK RESPONSE TIME PF'][i] = 0
            sleep(2)




            # FAILURE RATE

        try:
            driver.get(base_dados.iloc[i][11])

            sleep(15)

            element = driver.find_element_by_xpath('//*[@id="rootPageContentElementId"]/div/div[2]/div[1]/div[4]/div[2]/div/div[1]/div[1]/div/div/div/div/div//table')
            html_content = element.get_attribute('outerHTML')

            # limpando dados
            soup = BeautifulSoup(html_content, 'html.parser')
            table = soup.find(name='table')

            # criando base de dados
            base_dyna = pd.read_html(str(table))[0]

            if len(base_dyna) > 2:
                erros4 = soma_valores_taxaErros(base_dyna)
                base_dados['LINK FAILURE RATE CB'][i] = erros4
                sleep(2)

            if len(base_dyna) < 3:
                base_dados['LINK FAILURE RATE CB'][i] = base_dyna['Failure rate'][0]
                sleep(2)


        except:
            base_dados['LINK FAILURE RATE CB'][i] = 0
            sleep(2)

    


        try:
            driver.get(base_dados.iloc[i][12])

            sleep(15)

            element = driver.find_element_by_xpath('//*[@id="rootPageContentElementId"]/div/div[2]/div[1]/div[4]/div[2]/div/div[1]/div[1]/div/div/div/div/div//table')
            html_content = element.get_attribute('outerHTML')

            # limpando dados
            soup = BeautifulSoup(html_content, 'html.parser')
            table = soup.find(name='table')

            # criando base de dados
            base_dyna = pd.read_html(str(table))[0]

            if len(base_dyna) > 2:
                erros5 = soma_valores_responseTime(base_dyna)
                base_dados['LINK FAILURE RATE EX'][i] = erros5
                sleep(2)

            if len(base_dyna) < 3:
                base_dados['LINK FAILURE RATE EX'][i] = base_dyna['Failure rate'][0]
                sleep(2)

        
        except:
            base_dados['LINK FAILURE RATE EX'][i] = 0
            sleep(2)

        


        try:    
            driver.get(base_dados.iloc[i][13])

            sleep(15)

            element = driver.find_element_by_xpath('//*[@id="rootPageContentElementId"]/div/div[2]/div[1]/div[4]/div[2]/div/div[1]/div[1]/div/div/div/div/div//table')
            html_content = element.get_attribute('outerHTML')

            # limpando dados
            soup = BeautifulSoup(html_content, 'html.parser')
            table = soup.find(name='table')

            # criando base de dados
            base_dyna = pd.read_html(str(table))[0]

            if len(base_dyna) > 2:
                erros6 = soma_valores_responseTime(base_dyna)
                base_dados['LINK FAILURE RATE PF'][i] = erros6
                sleep(2)

            if len(base_dyna) < 3:
                base_dados['LINK FAILURE RATE PF'][i] = base_dyna['Failure rate'][0]
                sleep(2)

        except:
            base_dados['LINK FAILURE RATE PF'][i] = 0
            sleep(2)




        # CPU

        try:
            driver.get(base_dados.iloc[i][14])

            sleep(15)

            element = driver.find_element_by_xpath('//*[@id="rootPageContentElementId"]/div/div[2]/div[1]/div[4]/div[2]/div/div[1]/div[1]/div/div/div/div/div//table')
            html_content = element.get_attribute('outerHTML')

            # limpando dados
            soup = BeautifulSoup(html_content, 'html.parser')
            table = soup.find(name='table')

            # criando base de dados
            base_dyna = pd.read_html(str(table))[0]


            if len(base_dyna) > 2:
                cpu7 = soma_valores_CPU(base_dyna)
                base_dados['LINK CPU CB'][i] = cpu7
                sleep(2)

            else:
                base_dados['LINK CPU CB'][i] = base_dyna['CPU'][0]
                sleep(2)



        except:
            base_dados['LINK CPU CB'][i] = 0
            sleep(2)

        
        try:
            driver.get(base_dados.iloc[i][15])

            sleep(15)

            element = driver.find_element_by_xpath('//*[@id="rootPageContentElementId"]/div/div[2]/div[1]/div[4]/div[2]/div/div[1]/div[1]/div/div/div/div/div//table')
            html_content = element.get_attribute('outerHTML')

            # limpando dados
            soup = BeautifulSoup(html_content, 'html.parser')
            table = soup.find(name='table')

            # criando base de dados
            base_dyna = pd.read_html(str(table))[0]

            if len(base_dyna) > 2:
                cpu8 = soma_valores_CPU(base_dyna)
                base_dados['LINK CPU EX'][i] = cpu8
                sleep(2)

            else:
                base_dados['LINK CPU EX'][i] = base_dyna['CPU'][0]
                sleep(2)

            

        except:
            base_dados['LINK CPU EX'][i] = 0
            sleep(2)


        try:
            driver.get(base_dados.iloc[i][16])

            sleep(15)

            element = driver.find_element_by_xpath('//*[@id="rootPageContentElementId"]/div/div[2]/div[1]/div[4]/div[2]/div/div[1]/div[1]/div/div/div/div/div//table')
            html_content = element.get_attribute('outerHTML')

            # limpando dados
            soup = BeautifulSoup(html_content, 'html.parser')
            table = soup.find(name='table')

            # criando base de dados
            base_dyna = pd.read_html(str(table))[0]

            if len(base_dyna) > 2:
                cpu9 = soma_valores_CPU(base_dyna)
                base_dados['LINK CPU PF'][i] = cpu9
                sleep(2)

            else:
                base_dados['LINK CPU PF'][i] = base_dyna['CPU'][0]
                sleep(2)

        except:
            base_dados['LINK CPU PF'][i] = 0
            sleep(2)



    

        # REQUESTS

        try:
            driver.get(base_dados.iloc[i][17])

            sleep(15)

            element = driver.find_element_by_xpath('//*[@id="rootPageContentElementId"]/div/div[2]/div[1]/div[4]/div[2]/div/div[1]/div[1]/div/div/div/div/div//table')
            html_content = element.get_attribute('outerHTML')

            # limpando dados
            soup = BeautifulSoup(html_content, 'html.parser')
            table = soup.find(name='table')

            # criando base de dados
            base_dyna = pd.read_html(str(table))[0]

            if len(base_dyna) > 2:
                request10 = soma_valores_Requests(base_dyna)
                base_dados['LINK REQUESTS CB'][i] = request10
                sleep(2)

            else:
                base_dados['LINK REQUESTS CB'][i] = base_dyna['Processed requests'][0]
                sleep(2)


        except:
            base_dados['LINK REQUESTS CB'][i] = 0
            sleep(2)



        try:
            driver.get(base_dados.iloc[i][18])

            sleep(15)

            element = driver.find_element_by_xpath('//*[@id="rootPageContentElementId"]/div/div[2]/div[1]/div[4]/div[2]/div/div[1]/div[1]/div/div/div/div/div//table')
            html_content = element.get_attribute('outerHTML')

            # limpando dados
            soup = BeautifulSoup(html_content, 'html.parser')
            table = soup.find(name='table')

            # criando base de dados
            base_dyna = pd.read_html(str(table))[0]

            if len(base_dyna) > 2:
                request11 = soma_valores_Requests(base_dyna)
                base_dados['LINK REQUESTS EX'][i] = request11
                sleep(2)

            else:
                base_dados['LINK REQUESTS EX'][i] = base_dyna['Processed requests'][0]
                sleep(2)


        except:
            base_dados['LINK REQUESTS EX'][i] = 0
            sleep(2)



        try:
            driver.get(base_dados.iloc[i][19])

            sleep(15)

            element = driver.find_element_by_xpath('//*[@id="rootPageContentElementId"]/div/div[2]/div[1]/div[4]/div[2]/div/div[1]/div[1]/div/div/div/div/div//table')
            html_content = element.get_attribute('outerHTML')

            # limpando dados
            soup = BeautifulSoup(html_content, 'html.parser')
            table = soup.find(name='table')

            # criando base de dados
            base_dyna = pd.read_html(str(table))[0]

            if len(base_dyna) > 2:
                request12 = soma_valores_Requests(base_dyna)
                base_dados['LINK REQUESTS PF'][i] = request12
                sleep(2)

            else:
                base_dados['LINK REQUESTS PF'][i] = base_dyna['Processed requests'][0]
                sleep(2)

        

        except:
            base_dados['LINK REQUESTS PF'][i] = 0
            sleep(2)

else:
    pass

In [ ]:
bateria_3 = base_dados.copy()

In [ ]:
# LIMPEZA DE DADOS NA BATERIA 3

bateria_3['LINK RESPONSE TIME CB'] = limpeza_responseTime(bateria_3['LINK RESPONSE TIME CB'])

bateria_3['LINK RESPONSE TIME EX'] = limpeza_responseTime(bateria_3['LINK RESPONSE TIME EX'])

bateria_3['LINK RESPONSE TIME PF'] = limpeza_responseTime(bateria_3['LINK RESPONSE TIME PF'])

############################################################################################################

bateria_3['LINK FAILURE RATE CB'] = limpeza_taxaErros(bateria_3['LINK FAILURE RATE CB'])

bateria_3['LINK FAILURE RATE EX'] = limpeza_taxaErros(bateria_3['LINK FAILURE RATE EX'])

bateria_3['LINK FAILURE RATE PF'] = limpeza_taxaErros(bateria_3['LINK FAILURE RATE PF'])

############################################################################################################

bateria_3['LINK CPU CB'] = limpeza_CPU(bateria_3['LINK CPU CB'])

bateria_3['LINK CPU EX'] = limpeza_CPU(bateria_3['LINK CPU EX'])

bateria_3['LINK CPU PF'] = limpeza_CPU(bateria_3['LINK CPU PF'])

############################################################################################################

bateria_3['LINK REQUESTS CB'] = limpeza_Requests(bateria_3['LINK REQUESTS CB'])

bateria_3['LINK REQUESTS EX'] = limpeza_Requests(bateria_3['LINK REQUESTS EX'])

bateria_3['LINK REQUESTS PF'] = limpeza_Requests(bateria_3['LINK REQUESTS PF'])

In [ ]:
# RENOMEANDO COLUNAS DO DATASET 
bateria_3 = bateria_3.rename(columns={'LINK RESPONSE TIME CB': 'Resultado CB Response Time ms', 'LINK RESPONSE TIME EX': 'Resultado EX Response Time ms', 'LINK RESPONSE TIME PF': 'Resultado PF Response Time ms', 'LINK FAILURE RATE CB': 'Resultado CB Taxa de Erros %', 'LINK FAILURE RATE EX': 'Resultado EX Taxa de Erros %', 'LINK FAILURE RATE PF':'Resultado PF Taxa de Erros %', 'LINK CPU CB': 'Resultado CB Consumo CPU %', 'LINK CPU EX': 'Resultado EX Consumo CPU %', 'LINK CPU PF': 'Resultado PF Consumo CPU %', 'LINK REQUESTS CB': 'Resultado CB Throughput Reqs/min', 'LINK REQUESTS EX': 'Resultado EX Throughput Reqs/min', 'LINK REQUESTS PF':'Resultado PF Throughput Reqs/min'})

In [ ]:
# FORMATANDO NÚMEROS PARA MANTIMENTO DE UM PADRÃO

for i in range(len(bateria_3)):
    try:
        bateria_3['Resultado CB Response Time ms'][i] = float(bateria_3['Resultado CB Response Time ms'][i])

        bateria_3['Resultado CB Response Time ms'][i] = float(f'{bateria_3["Resultado CB Response Time ms"][i]:.1f}')

    except:
        pass

    
    try:
        bateria_3['Resultado EX Response Time ms'][i] = float(bateria_3['Resultado EX Response Time ms'][i])

        bateria_3['Resultado EX Response Time ms'][i] = float(f'{bateria_3["Resultado EX Response Time ms"][i]:.1f}')

    except:
        pass


    try:
        bateria_3['Resultado PF Response Time ms'][i] = float(bateria_3['Resultado PF Response Time ms'][i])

        bateria_3['Resultado PF Response Time ms'][i] = float(f'{bateria_3["Resultado PF Response Time ms"][i]:.1f}')

    except:
        pass


    try:
        bateria_3['Resultado CB Taxa de Erros %'][i] = float(bateria_3['Resultado CB Taxa de Erros %'][i])

        bateria_3['Resultado CB Taxa de Erros %'][i] = float(f'{bateria_3["Resultado CB Taxa de Erros %"][i]:.1f}')

    except:
        pass


    try:
        bateria_3['Resultado EX Taxa de Erros %'][i] = float(bateria_3['Resultado EX Taxa de Erros %'][i])

        bateria_3['Resultado EX Taxa de Erros %'][i] = float(f'{bateria_3["Resultado EX Taxa de Erros %"][i]:.1f}')

    except:
        pass


    try:
        bateria_3['Resultado PF Taxa de Erros %'][i] = float(bateria_3['Resultado PF Taxa de Erros %'][i])

        bateria_3['Resultado PF Taxa de Erros %'][i] = float(f'{bateria_3["Resultado PF Taxa de Erros %"][i]:.1f}')

    except:
        pass



    try:
        bateria_3['Resultado CB Consumo CPU %'][i] = float(bateria_3['Resultado CB Consumo CPU %'][i])

        bateria_3['Resultado CB Consumo CPU %'][i] = float(f'{bateria_3["Resultado CB Consumo CPU %"][i]:.1f}')

    except:
        pass


    try:
        bateria_3['Resultado EX Consumo CPU %'][i] = float(bateria_3['Resultado EX Consumo CPU %'][i])

        bateria_3['Resultado EX Consumo CPU %'][i] = float(f'{bateria_3["Resultado EX Consumo CPU %"][i]:.1f}')

    except:
        pass


    try:
        bateria_3['Resultado PF Consumo CPU %'][i] = float(bateria_3['Resultado PF Consumo CPU %'][i])

        bateria_3['Resultado PF Consumo CPU %'][i] = float(f'{bateria_3["Resultado PF Consumo CPU %"][i]:.1f}')

    except:
        pass



    try:
        bateria_3['Resultado CB Throughput Reqs/min'][i] = float(bateria_3['Resultado CB Throughput Reqs/min'][i])

        bateria_3['Resultado CB Throughput Reqs/min'][i] = float(f'{bateria_3["Resultado CB Throughput Reqs/min"][i]:.1f}')

    except:
        pass



    try:
        bateria_3['Resultado EX Throughput Reqs/min'][i] = float(bateria_3['Resultado EX Throughput Reqs/min'][i])

        bateria_3['Resultado EX Throughput Reqs/min'][i] = float(f'{bateria_3["Resultado EX Throughput Reqs/min"][i]:.1f}')

    except:
        pass


    try:
        bateria_3['Resultado PF Throughput Reqs/min'][i] = float(bateria_3['Resultado PF Throughput Reqs/min'][i])

        bateria_3['Resultado PF Throughput Reqs/min'][i] = float(f'{bateria_3["Resultado PF Throughput Reqs/min"][i]:.1f}')

    except:
        pass



# 5.0 Exportação de Dados

## 5.1 Unindo Dados das Baterias do Teste

In [63]:
# CONCATENANDO A BASE DE DADOS DE ACORDO COM O NÚMERO DE BATERIAS QUE TIVERAM NO TESTE

if int(num_bateria) == 1:
    pass

if int(num_bateria) == 2:

    B2B_Automacao_Dyna = pd.concat([bateria_1, bateria_2], ignore_index=True)

if int(num_bateria) == 3:

    B2B_Automacao_Dyna = pd.concat([bateria_1, bateria_2, bateria_3], ignore_index=True)

## 5.2 Exportando Base Final

In [64]:
B2B_Automacao_Dyna.to_excel(r'C:\Users\Andre\Documents\Via\Extracao_Automatica\B2B_Automacao_Dynatrace16-09-21.xlsx', index=False)